# Homework One_A/B testing 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest

In [2]:
df = pd.read_csv("Algorithm_data.csv")
df.head()

,Conversion,Group,Algorithm
0,0,control,Popularity
1,0,treatment,Personalzied
2,0,control,Popularity
3,0,control,Popularity
4,0,control,Popularity


## 2 a) What is our prior estimate of the variance of the OEC ($σ^2$)?

In [4]:
p = .05
variance = p*(1-p)
print(f'The variance of the OEC is {variance}.')

The variance of the OEC is 0.0475.


## 2 b) How does the number of samples needed change depending on the sensitivity? 

In [9]:
def power_simple(variance, sensitivity):
    n = (16*variance)/sensitivity**2
    return(int(n))
for sen in np.arange(0.01,0.06,0.01):
    print(sen,power_simple(variance,sen))
print('As the sensitivity became larger, the number of samples needed became smaller.')

0.01 7600
0.02 1900
0.03 844
0.04 475
0.05 303
As the sensitivity became larger, the number of samples needed became smaller.


## 2 c) What should our minimum sample size be for this experiment with Δ = 3%? 

In [11]:
print(f'The minimum sample size would be {power_simple(variance,0.03)} when sensitivy is 3%.')

The minimum sample size would be 844 when sensitivy is 3%.


## 2 d)  What would be our required sample size using this metric? What does this tell us about choosing OECs with larger or smaller variances?

standard deviation = $\sqrt{p*(1-p)}$ = 30  
$\Delta$=3%  
$\sigma$ = $\sqrt{p*(1-p)*(\frac{1}{n_c}+\frac{1}{n_e})}$  
$n_c$=$n_e$=n=$\frac{16\sigma^2}{\Delta^2}$  
so we came up with an equation:  
n=$\frac{16*p*(1-p)*(\frac{1}{n}+\frac{1}{n})}{\Delta^2}$  
which means  
n=$\frac{4*standard\ deviation*\sqrt{2}}{\Delta}$

In [23]:
n_sample=round(120*np.sqrt(2)/0.03)
print(f"So the minimum sample size would be {n_sample}.\nAnd we can came to a conclusion that required sample size will become larger if we choose OEC with larger variances.")

So the minimum sample size would be 5657.
And we can came to a conclusion that required sample size will become larger if we choose OEC with larger variances.


## 3 a) Test whether the difference between the control and treatment conversion rates is statistically significant. Let α = 5%.

In [22]:
pd.crosstab(df.Algorithm,df.Group)

Group,control,treatment
Algorithm,,
Personalzied,0,51311
Popularity,51311,0


In [28]:
control_sample = df[df['Group'] == 'control'].sample(n=n_sample, random_state=22)
treatment_sample = df[df['Group'] == 'treatment'].sample(n=n_sample, random_state=10)
ab_test = pd.concat([control_sample, treatment_sample], axis=0)
ab_test.reset_index(drop=True, inplace=True)
ab_test.head()

,Conversion,Group,Algorithm
0,0,control,Popularity
1,0,control,Popularity
2,0,control,Popularity
3,0,control,Popularity
4,0,control,Popularity


In [40]:
control = control_sample['Conversion']
treatment = treatment_sample['Conversion']

# Get the counts fopr each group (is equal in this case)
n_c = len(control)
n_e = len(treatment)
diff = treatment.mean()-control.mean()
p_hat  = ab_test['Conversion'].mean()
sigma_d = np.sqrt(p_hat*(1-p_hat)*((1/n_c)+(1/n_e)))
t = diff/sigma_d
print(t)

7.548420423117743


In [41]:
successes = [ treatment.sum(),control.sum(),]
nobs = [n_c, n_e]


z_stat, pval = proportions_ztest(successes, nobs=nobs)
print(z_stat)

7.548420423117743


In [42]:
stats.ttest_ind(treatment,control)

Ttest_indResult(statistic=7.566830999078732, pvalue=4.121071852317913e-14)

From the tests above, we can't conclude that there is a meaningful difference between groups.

## 3 b) 

The hypothesis test is not enough to decide to implement the new algorithm because in business cases we also need to consider the economic significance. Even if the hypothesis test shows that there is a statistical significance between two groups, we will not implement the new algorithm if the cost is larger than the potential profits.

## 4 a)  95% confidence interval around the difference between the OECs of the treatment and control groups

In [43]:
upper = diff + 1.96*sigma_d
lower = diff - 1.96*sigma_d
print(f'ci 95% for difference: [{lower:.3f}, {upper:.3f}]')

ci 95% for difference: [0.026, 0.045]


## 4 b) economic significance threshold of 3%. 

Since the economic significance threshold 3% is within the 95% ci, we cannot make the decision whether to implement the new algorithm. And in reality, this uncertainty means we won't implement the new change.

## 4 c) 

If the economic significance level is above the upper bound of your confidence interval, I will not implement the new algorithm change.  
If the economic significance level is within your confidence interval,It is not clear whether it is worth it to implement the change. So we need to have more testing (possibly with a larger sample).

## 5 a) 

In [45]:
print(f"{np.power(2,3)} user groups would be needed for a full factorial design with three factors.")

8 user groups would be needed for a full factorial design with three factors.


## 5 b) 

In [49]:
r = 2
p = .05  
sigma = np.sqrt(p*(1-p))
delta = .03
n = int(np.ceil(((4*r*sigma)/.02)**2))
print(f"The number of samples needed to run this experiment change if we used 3 levels (two treatment groups) for each factor instead of just one is {n}.")

The number of samples needed to run this experiment change if we used 3 levels (two treatment groups) for each factor instead of just one is 7600.


## 5 c) 

In [64]:
conversion_df=pd.DataFrame()
conversion_df['x1']=[1,1,0,0]
conversion_df['x2']=[1,0,1,0]
conversion_df['x1:x2']=conversion_df['x1']*conversion_df['x2']
beta0=-1
beta1=0.15
beta2=0.08
beta3=-0.1
conversion_df['conversion p']=np.exp(beta0+beta1*conversion_df['x1']+beta2*conversion_df['x2']+beta3*conversion_df['x1:x2'])/(1+np.exp(beta0+beta1*conversion_df['x1']+beta2*conversion_df['x2']+beta3*conversion_df['x1:x2']))
print(f'The combination that yields the highest probability of conversion is:\n{conversion_df.max(axis=0)}.\nThe highest probability of conversion is {round(conversion_df.max(axis=0)[3],4)}')

The combination that yields the highest probability of conversion is:
x1              1.000000
x2              1.000000
x1:x2           1.000000
conversion p    0.299433
dtype: float64.
The highest probability of conversion is 0.2994
